In [1]:
!nvidia-smi

Thu Jun 17 00:11:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sameershaikzz","key":"7933cf2548d52ca2f0030d729b88f6e4"}'}

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
! pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 61kB 5.2MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=e423b64a72a39b9b28e2651dc9649a1a6d1065b4eec556cdf1170fdd2a6b188a
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [5]:
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle competitions download -c deep-learning-hw2p2-part-2-classification

100% 591M/594M [00:11<00:00, 79.3MB/s]
100% 594M/594M [00:11<00:00, 52.1MB/s]


In [7]:
!unzip \*.zip  && rm *.zip


Streaming output truncated to the last 5000 lines.
  inflating: val_data/val_data/2348/0001_01.jpg  
  inflating: val_data/val_data/2348/0046_01.jpg  
  inflating: val_data/val_data/2349/0082_01.jpg  
  inflating: val_data/val_data/2349/0312_02.jpg  
  inflating: val_data/val_data/235/0008_02.jpg  
  inflating: val_data/val_data/235/0140_03.jpg  
  inflating: val_data/val_data/2350/0106_01.jpg  
  inflating: val_data/val_data/2350/0339_01.jpg  
  inflating: val_data/val_data/2351/0006_01.jpg  
  inflating: val_data/val_data/2351/0150_01.jpg  
  inflating: val_data/val_data/2352/0193_01.jpg  
  inflating: val_data/val_data/2352/0311_01.jpg  
  inflating: val_data/val_data/2353/0229_01.jpg  
  inflating: val_data/val_data/2353/0297_02.jpg  
  inflating: val_data/val_data/2354/0140_01.jpg  
  inflating: val_data/val_data/2354/0444_01.jpg  
  inflating: val_data/val_data/2355/0112_02.jpg  
  inflating: val_data/val_data/2355/0385_01.jpg  
  inflating: val_data/val_data/2356/0097_01.jpg  
 

In [ ]:
#!unzip \*.zip  -d /content/drive/MyDrive/Homeworks/HW2P2

In [27]:
import numpy as np
import torch
import PIL
import torchvision   

from torch.utils.data import Dataset, DataLoader

In [28]:
import os
import numpy as np
from PIL import Image

import torch
import torchvision   
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

In [29]:
import os
from PIL import Image

In [30]:
def parse_data(datadir):
    img_list = []
    ID_list = []
    for root, directories, filenames in os.walk(datadir):  #root: median/1
        for filename in filenames:
            if filename.endswith('.jpg'):
                filei = os.path.join(root, filename)
                img_list.append(filei)
                ID_list.append(root.split('/')[-1])
                
                # print(ID_list)

    # construct a dictionary, where key and value correspond to ID and target
    uniqueID_list = list(set(ID_list))
    class_n = len(uniqueID_list)
    target_dict = dict(zip(uniqueID_list, range(class_n)))
    target_num= dict(zip( range(class_n),uniqueID_list))
    print(target_num)
    print(target_dict)
    label_list = [target_dict[ID_key] for ID_key in ID_list]
    print(len(label_list))

    print('{}\t\t{}\n{}\t\t{}'.format('#Images', '#Labels', len(img_list), len(set(label_list))))
    return img_list, label_list, class_n,target_num


In [31]:
img_list, label_list, class_n,target_num = parse_data('/content/train_data/train_data')
print(len(img_list),len(label_list),class_n)

{0: '3308', 1: '995', 2: '2664', 3: '2458', 4: '2141', 5: '3527', 6: '1387', 7: '3510', 8: '756', 9: '1051', 10: '2602', 11: '513', 12: '3504', 13: '2899', 14: '641', 15: '1083', 16: '2753', 17: '1780', 18: '217', 19: '3940', 20: '181', 21: '2315', 22: '987', 23: '933', 24: '1196', 25: '3493', 26: '1365', 27: '1774', 28: '3989', 29: '1913', 30: '1676', 31: '3889', 32: '3550', 33: '2627', 34: '1372', 35: '1745', 36: '3187', 37: '1213', 38: '764', 39: '2014', 40: '3737', 41: '1052', 42: '2551', 43: '1013', 44: '542', 45: '2848', 46: '3149', 47: '447', 48: '1677', 49: '847', 50: '1651', 51: '517', 52: '410', 53: '1954', 54: '2969', 55: '173', 56: '3963', 57: '3543', 58: '2873', 59: '3097', 60: '2986', 61: '108', 62: '3907', 63: '3500', 64: '1566', 65: '1274', 66: '677', 67: '1554', 68: '3491', 69: '116', 70: '973', 71: '1826', 72: '84', 73: '3934', 74: '2120', 75: '2588', 76: '2404', 77: '1392', 78: '3961', 79: '3186', 80: '3051', 81: '862', 82: '2255', 83: '3327', 84: '2160', 85: '1215',

In [32]:
val_img_list, val_label_list, class_n,target_num = parse_data('/content/val_data/val_data')
#a_file = open("data.pkl", "wb")
#pickle.dump(target_num, a_file)
#a_file.close()
print(len(val_img_list),len(val_label_list),class_n)

{0: '3308', 1: '995', 2: '2664', 3: '2458', 4: '2141', 5: '3527', 6: '1387', 7: '3510', 8: '756', 9: '1051', 10: '2602', 11: '513', 12: '3504', 13: '2899', 14: '641', 15: '1083', 16: '2753', 17: '1780', 18: '217', 19: '3940', 20: '181', 21: '2315', 22: '987', 23: '933', 24: '1196', 25: '3493', 26: '1365', 27: '1774', 28: '3989', 29: '1913', 30: '1676', 31: '3889', 32: '3550', 33: '2627', 34: '1372', 35: '1745', 36: '3187', 37: '1213', 38: '764', 39: '2014', 40: '3737', 41: '1052', 42: '2551', 43: '1013', 44: '542', 45: '2848', 46: '3149', 47: '447', 48: '1677', 49: '847', 50: '1651', 51: '517', 52: '410', 53: '1954', 54: '2969', 55: '173', 56: '3963', 57: '3543', 58: '2873', 59: '3097', 60: '2986', 61: '108', 62: '3907', 63: '3500', 64: '1566', 65: '1274', 66: '677', 67: '1554', 68: '3491', 69: '116', 70: '973', 71: '1826', 72: '84', 73: '3934', 74: '2120', 75: '2588', 76: '2404', 77: '1392', 78: '3961', 79: '3186', 80: '3051', 81: '862', 82: '2255', 83: '3327', 84: '2160', 85: '1215',

In [33]:
class ImageDataset(Dataset):
    def __init__(self, file_list, target_list,transform):
        self.file_list = file_list
        self.target_list = target_list
        self.n_class = len(list(set(target_list)))
        self.transform=transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img = Image.open(self.file_list[index])
        img = self.transform(img)
        #img = torchvision.transforms.ToTensor()(img)
        label = self.target_list[index]
        return img, label


   

    

In [48]:
from torchvision import transforms

In [49]:

transforms = transforms.Compose([
    
    transforms.RandomHorizontalFlip(),   
    torchvision.transforms.RandomVerticalFlip(),
    transforms.RandomRotation(15),
    
    transforms.ToTensor()
    
    ])

In [50]:
trainset = ImageDataset(img_list, label_list,transform=transforms)

In [51]:
dataloader = DataLoader(trainset, batch_size=256, shuffle=True, num_workers=1, drop_last=False)

In [52]:
train_data_item, train_data_label = trainset.__getitem__(73180)
print(train_data_label,train_data_item)

2597 tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])


In [53]:
class ImageDataset(Dataset):
    def __init__(self, file_list, target_list):
        self.file_list = file_list
        self.target_list = target_list
        self.n_class = len(list(set(target_list)))
        

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img = Image.open(self.file_list[index])
        #img = self.transform(img)
        img = torchvision.transforms.ToTensor()(img)
        label = self.target_list[index]
        return img, label


In [54]:
valset= ImageDataset(val_img_list,val_label_list)

In [55]:
dev_dataloader= DataLoader(valset, batch_size=256, shuffle=False, num_workers=1)

In [56]:
val_data_item, val_data_label = trainset.__getitem__(7180)
print(val_data_label,val_data_item)

540 tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0039, 0.0039, 0.0039],
         ...,
         [0.0078, 0.0118, 0.0118,  ..., 0.0196, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0549, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.1882, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0039, 0.0039, 0.0039],
         ...,
         [0.0118, 0.0118, 0.0118,  ..., 0.0078, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0353, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.1765, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0039, 0.0039

In [57]:
print('data item shape: {}\t data item label: {}'.format(train_data_item.shape, train_data_label))

data item shape: torch.Size([3, 64, 64])	 data item label: 2597


In [ ]:
"""
class SimpleResidualBlock(nn.Module):
    def __init__(self, channel_size,channel_next, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(channel_next, channel_next, kernel_size=3,padding=1, stride=stride, bias=False)
        #self.bn11 = nn.BatchNorm2d(channel_next)
        #self.conv2 = nn.Conv2d(channel_next, channel_next, kernel_size=3,padding=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(channel_next)
        self.relu1 = nn.ReLU()
        if stride == 1:
            self.shortcut = nn.Conv2d(channel_next, channel_next, kernel_size=3,padding=1, stride=stride, bias=False)
            self.bn2 = nn.BatchNorm2d(channel_next)
            #self.relu2 = nn.ReLU()
            #self.bn2 = nn.BatchNorm2d(channel_next)
        else:
            
            self.shortcut = nn.Conv2d(channel_size, channel_next, kernel_size=1, stride=2)
            self.bn2 = nn.BatchNorm2d(channel_next)
            #self.relu2 = nn.ReLU()
        #self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.conv1(x)
        out=self.bn1(out) 
        #out = self.conv2(out)
        #out = self.bn1(out)
        out=self.relu1(out)
        shortcut = self.shortcut(x)

        bbnn = self.bn2(shortcut)
        #bbnn =self.relu2(bbnn)

        
        out =out + shortcut+bbnn
        
        return out
"""

In [ ]:
"""
class ClassificationNetwork(nn.Module):
    def __init__(self, in_features, num_classes):
        super().__init__()
        
        self.layers = nn.Sequential(
            nn.Conv2d(in_features, 64, kernel_size=3, stride=1,padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
           # SimpleResidualBlock(64,64,1), #k=4,pad=2  ==set  okkk  64
            
            SimpleResidualBlock(64,64,1),   # k=4,pad=2  =  64
            #maxpool + conv2d
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),       # changed stride
            #SimpleResidualBlock(64,128,2),
            nn.Conv2d(64,128,kernel_size=1, stride=1),            # changed stride
            nn.BatchNorm2d(128),
            #SimpleResidualBlock(64,128,1), #k=4,pad=2    pad =2 32
            
            SimpleResidualBlock(128,128,1),#k=4,pad=2 65  32
             nn.BatchNorm2d(128),
            #SimpleResidualBlock(128,256,2),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),     # changed stride
           
            nn.Dropout(0.1),
            nn.Conv2d(128,256,kernel_size=1, stride=1),
             nn.BatchNorm2d(256),           # changed stride
           

            #SimpleResidualBlock(128,256,1),#k=4,pad=2 16
            
            #SimpleResidualBlock(256,256,2), 
            #nn.Conv2d(in_features, 64, kernel_size=3, stride=1,padding=1, bias=False),
            SimpleResidualBlock(256,256,1),
            nn.BatchNorm2d(256),#k=4,pad=2=4,pad=2 4
            #nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Dropout(0.1),
            #SimpleResidualBlock(256,512,2),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),   # changed stride
            nn.Conv2d(256,512,kernel_size=1, stride=1),           # changed stride
            
            nn.BatchNorm2d(512),
            SimpleResidualBlock(512,512,1),
            #nn.Dropout(0.1),
            #SimpleResidualBlock(512,512,2), # 2
            
            nn.AdaptiveAvgPool2d((1, 1)), # For each channel, collapses (averages) the entire feature map (height & width) to 1x1
            nn.Flatten(),
            
             # the above ends up with batch_size x 512 x 1 x 1, 
        )
        
        self.linear= nn.Linear(512, 4000)
        #self.linear = nn.Linear(4500, 4000)
        # add layer

       
      
        
    def forward(self, x, return_embedding=False):
       
        embedding = self.layers(x) 
        
        if return_embedding:
            return embedding
        else:
          
          
          return self.linear(embedding)



"""


In [96]:
class SimpleResidualBlock(nn.Module):
    def __init__(self, channel_size,channel_next, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(channel_size, channel_next, kernel_size=3,padding=1, stride=stride, bias=False) 
        #self.conv2 = nn.Conv2d(channel_next, channel_next, kernel_size=3,padding=1, stride=1, bias=False)        
        self.bn1 = nn.BatchNorm2d(channel_next)
        if stride == 1:
            self.shortcut = nn.Identity()
        else:
            self.shortcut = nn.Conv2d(channel_size, channel_next, kernel_size=1, stride=stride)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.conv1(x)
        #out=self.conv2(x)
        out = self.bn1(out)
        
        shortcut = self.shortcut(x)
        
        out = self.relu(out + shortcut)
        
        return out

In [101]:
class ClassificationNetwork(nn.Module):
    def __init__(self, in_features, num_classes):
        super().__init__()
        
        self.layers = nn.Sequential(
            nn.Conv2d(in_features, 64, kernel_size=3, stride=1,padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            SimpleResidualBlock(64,64,1),
            SimpleResidualBlock(64,128,2),
            SimpleResidualBlock(128,128,1),
            SimpleResidualBlock(128,256,2),
            SimpleResidualBlock(256,256,1),
            SimpleResidualBlock(256,512,2),
            SimpleResidualBlock(512,512,1),
            nn.Conv2d(512,1024,kernel_size=1, stride=1),
            nn.AdaptiveAvgPool2d((1, 1)), # For each channel, collapses (averages) the entire feature map (height & width) to 1x1
            nn.Flatten(), # the above ends up with batch_size x 64 x 1 x 1, flatten to batch_size x 64
            
        )
        self.linear = nn.Linear(1024, num_classes)
        
    def forward(self, x, return_embedding=False):
        embedding = self.layers(x) 
        
        if return_embedding:
            return embedding
        else:
            return self.linear(embedding)


In [102]:
from torch.optim.lr_scheduler import StepLR

in_features = 3 # RGB channels
#learningRate = 0.01
learningRate = 0.025
#learningRate = 0.01
#learningRate = 1e-2
weightDecay = 5e-5
#weightDecay = 0.01
num_classes = 4000

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

network = ClassificationNetwork(in_features, num_classes)
#network=ResNet9(in_features, num_classes)
network = network.to(device)
#criterion = CenterLoss()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network.parameters(),lr=learningRate,weight_decay=weightDecay,momentum=0.9)

#scheduler = StepLR(optimizer, step_size=1, gamma=0.1)
print(network)

ClassificationNetwork(
  (layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): SimpleResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Identity()
      (relu): ReLU()
    )
    (4): SimpleResidualBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2))
      (relu): ReLU()
    )
    (5): SimpleResidualBlock(
      (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0

In [103]:
# Train!
scheduler = StepLR(optimizer, step_size=20, gamma=0.04)
def train_epoch(network,dataloader,criterion,optimizer,epoch):
    # Train
    network.train()
    avg_loss = 0.0
    num_correct=0
    total_predictions=0
    accuracy=0
    num_total=0
    running_loss=0
    accur=[]

    for batch_num, (x, y) in enumerate(dataloader):
        optimizer.zero_grad()
        
        x, y = x.to(device), y.to(device)

        outputs = network(x)

        loss = criterion(outputs, y.long())
        loss.backward()
        optimizer.step()

        avg_loss += loss.item()
        value,index=torch.max(outputs,1)
        num_correct=torch.sum(index==y)
        total_predictions=outputs.shape[0]
        # print("Number of correct predictions",num_correct)
        num_total=num_total+num_correct
        batch_accuracy=num_correct/256
        # print(batch_num,"bactch accuracy is:",batch_accuracy)
        # running_loss/=len(dataloader)
        # print("Training Loss:", running_loss)


        torch.cuda.empty_cache()
        del x
        del y
        del loss
    


        # if batch_num % 10 == 9:
        #     print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch, batch_num+1, avg_loss/))
        #     avg_loss = 0.0
    
    # # Validate
    # network.eval()
    # num_correct = 0
    # for batch_num, (x, y) in enumerate(dev_dataloader):
    #     x, y = x.to(device), y.to(device)
    #     outputs = network(x)
    #     num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()
        
    # print('Epoch: {}, Validation Accuracy: {:.2f}'.format(epoch, num_correct / len(dev_dataset)))
    print("*****************************************************++++++++++++++")
    accuracy=num_total/380638
    accur.append(accuracy)
    print(epoch,"epoch: Training accuracy",accuracy)
    print("****************************************************++++++++++++++")
    path = '/content/drive/MyDrive/Homeworks/HW2P2/Models/'+str(epoch)
    torch.save(network.state_dict(), path)
    # Validate
    



In [104]:


def val_epoch(network,dev_dataloader):
  network.eval()
  avg_loss = 0.0
  num_correct=0
  total_predictions=0
  accuracy=0
  num_total=0
  running_loss=0
  accur=[]

  for batch_num, (x, y) in enumerate(dev_dataloader):
      
      x, y = x.to(device), y.to(device)

      outputs = network(x)

    
      # print(y)

      
      value,index=torch.max(outputs,1)
      num_correct=torch.sum(index==y)
      # print(y)
      # print(index)
      total_predictions=outputs.shape[0]
      print("Number of correct predictions",num_correct)
      num_total=num_total+num_correct
      batch_accuracy=num_correct/256
      # print(batch_num,"bactch accuracy is:",batch_accuracy)
      # running_loss/=len(dataloader)
      # print("Training Loss:", running_loss)


      torch.cuda.empty_cache()
      del x
      del y



      # if batch_num % 10 == 9:
      #     print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch, batch_num+1, avg_loss/))
      #     avg_loss = 0.0

  # # Validate
  # network.eval()
  # num_correct = 0
  # for batch_num, (x, y) in enumerate(dev_dataloader):
  #     x, y = x.to(device), y.to(device)
  #     outputs = network(x)
  #     num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()
      
  # print('Epoch: {}, Validation Accuracy: {:.2f}'.format(epoch, num_correct / len(dev_dataset)))
  print("******************************************++++++++++++++")
  accuracy=num_total/8000
  print("val accuracy is:",accuracy)
  # accur.append(accuracy)
  # print(epoch,"epoch:",accuracy)
  # path = '/content/drive/MyDrive/Sujithafolder/CNN-Model'+str(epoch)
  # torch.save(network.state_dict(), path)
  # # Validate




In [105]:
nu_epoch=400
for epoch in range(320,nu_epoch):
  scheduler.step()
  # Print Learning Rate
  print('Epoch:', epoch,'LR:', scheduler.get_lr())
  train_epoch(network,dataloader,criterion,optimizer,epoch)
  val_epoch(network,dev_dataloader)

Epoch: 320 LR: [0.025]


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:370: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


*****************************************************++++++++++++++
320 epoch: Training accuracy tensor(0.0067, device='cuda:0')
****************************************************++++++++++++++
Number of correct predictions tensor(2, device='cuda:0')
Number of correct predictions tensor(2, device='cuda:0')
Number of correct predictions tensor(3, device='cuda:0')
Number of correct predictions tensor(5, device='cuda:0')
Number of correct predictions tensor(4, device='cuda:0')
Number of correct predictions tensor(5, device='cuda:0')
Number of correct predictions tensor(2, device='cuda:0')
Number of correct predictions tensor(2, device='cuda:0')
Number of correct predictions tensor(4, device='cuda:0')
Number of correct predictions tensor(5, device='cuda:0')
Number of correct predictions tensor(1, device='cuda:0')
Number of correct predictions tensor(2, device='cuda:0')
Number of correct predictions tensor(4, device='cuda:0')
Number of correct predictions tensor(6, device='cuda:0')
Number

KeyboardInterrupt: ignored

In [ ]:
# img1=Image.open('/content/val_data/val_data/3907/0360_05.jpg')
# plt.imshow(img1)


In [ ]:
# %cd /content/val_data/val_data/3907


In [ ]:
# ls

In [ ]:
# path='/content/drive/MyDrive/Sujithafolder/CNN-Model10'
# state_dict = torch.load(path)
# #print(len(state_dict["net.14.weight"]))
# network.load_state_dict(state_dict)
# # print(state_dict)
# # print(network)

# network.eval()
# val_num_correct = 0
# total_num_correct=0
# total_predictions=0
# bactchaccuracy=0
# for batch_num, (x, y) in enumerate(dev_dataloader):
#     x, y = x.to(device), y.to(device)
#     # print(x ,"===========================")
#     val_outputs = network(x)
#     value,index=torch.max(val_outputs,1)
#     val_num_correct=torch.sum(index==y)
#     print("correct predictions:",val_num_correct)
#     total_num_correct+=val_num_correct
#     total_predictions=val_outputs.shape[0]
#     bactchaccuracy=val_num_correct/total_predictions
#     print(bactchaccuracy)
# accuracy=total_num_correct/8000
# print("**************************",accuracy,"****************************")


In [ ]:
#  loading the model.

# path='/content/drive/MyDrive/Sujithafolder/CNN-Model10'
# state_dict = torch.load(path)
# #print(len(state_dict["net.14.weight"]))
# network.load_state_dict(state_dict)

In [ ]:
# val_epoch(network,dev_dataloader)

In [ ]:
state_dict = torch.load(path,map_location=torch.device('cpu'))
# #print(len(state_dict["net.14.weight"]))
network.load_state_dict(state_dict)


In [106]:

def parse_data(datadir):
    img_list = []
    ID_list = []
    for root, directories, filenames in os.walk(datadir):  #root: median/1
        for filename in filenames:
            if filename.endswith('.jpg'):
                filei = os.path.join(root, filename)
                img_list.append(filei)
                ID_list.append(root.split('/')[-1])
                # print(ID_list)

    # construct a dictionary, where key and value correspond to ID and target
    # uniqueID_list = list(set(ID_list))
    # class_n = len(uniqueID_list)
    # target_dict = dict(zip(uniqueID_list, range(class_n)))
    # print(target_dict)
    # label_list = [target_dict[ID_key] for ID_key in ID_list]
    # print(len(label_list))
    class_n=4000

    # print('{}\t\t{}\n{}\t\t{}'.format('#Images', '#Labels', len(img_list), len(set(label_list))))
    return img_list, class_n


In [107]:
# from PIL import Image


# img=Image.open('/content/val_data/val_data/3120/0001_01.jpg')
# plt.imshow(img)


class ImageDatatest(Dataset):
    def __init__(self, file_list):
        self.file_list = file_list
        # self.target_list = target_list
        # self.n_class = len(list(set(target_list)))
        self.n_class=4000

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img = Image.open(self.file_list[index])
        img = torchvision.transforms.ToTensor()(img)
        # label = self.target_list[index]
        return img



In [109]:
test_img_list, class_n = parse_data('/content/test_data/test_data')
testset = ImageDatatest(test_img_list)
test_dataloader= DataLoader(testset, batch_size=256, shuffle=False, num_workers=1)
test_data_item= testset.__getitem__(23)
print(target_num)

{0: '3308', 1: '995', 2: '2664', 3: '2458', 4: '2141', 5: '3527', 6: '1387', 7: '3510', 8: '756', 9: '1051', 10: '2602', 11: '513', 12: '3504', 13: '2899', 14: '641', 15: '1083', 16: '2753', 17: '1780', 18: '217', 19: '3940', 20: '181', 21: '2315', 22: '987', 23: '933', 24: '1196', 25: '3493', 26: '1365', 27: '1774', 28: '3989', 29: '1913', 30: '1676', 31: '3889', 32: '3550', 33: '2627', 34: '1372', 35: '1745', 36: '3187', 37: '1213', 38: '764', 39: '2014', 40: '3737', 41: '1052', 42: '2551', 43: '1013', 44: '542', 45: '2848', 46: '3149', 47: '447', 48: '1677', 49: '847', 50: '1651', 51: '517', 52: '410', 53: '1954', 54: '2969', 55: '173', 56: '3963', 57: '3543', 58: '2873', 59: '3097', 60: '2986', 61: '108', 62: '3907', 63: '3500', 64: '1566', 65: '1274', 66: '677', 67: '1554', 68: '3491', 69: '116', 70: '973', 71: '1826', 72: '84', 73: '3934', 74: '2120', 75: '2588', 76: '2404', 77: '1392', 78: '3961', 79: '3186', 80: '3051', 81: '862', 82: '2255', 83: '3327', 84: '2160', 85: '1215',

In [110]:
def test_epoch(network,test_dataloder):
  
  network.eval()
  avg_loss = 0.0
  num_correct=0
  total_predictions=0
  accuracy=0
  num_total=0
  running_loss=0
  accur=[]
  result=[]


  for batch_idx, data in enumerate(test_dataloader):

    data=data.to(device)
    outputs=network(data) 
    # print(outputs.shape)
    value,index=torch.max(outputs,1)
    result.append(index.tolist())
  return result 
      

      
      





In [111]:
result=test_epoch(network,test_dataloader)

In [112]:
images=[]
for i in range(len(test_img_list)):
  images.append(test_img_list[i].split("/")[-1])
print(images)

['7084.jpg', '7780.jpg', '4315.jpg', '5162.jpg', '5228.jpg', '5884.jpg', '383.jpg', '5352.jpg', '1666.jpg', '2134.jpg', '1441.jpg', '5085.jpg', '6195.jpg', '6824.jpg', '3358.jpg', '2517.jpg', '2793.jpg', '2485.jpg', '7827.jpg', '2306.jpg', '5380.jpg', '1148.jpg', '4603.jpg', '2497.jpg', '6695.jpg', '5992.jpg', '5101.jpg', '7110.jpg', '3318.jpg', '243.jpg', '2436.jpg', '1151.jpg', '5280.jpg', '5920.jpg', '7391.jpg', '915.jpg', '3876.jpg', '1410.jpg', '3100.jpg', '7905.jpg', '4673.jpg', '3675.jpg', '5764.jpg', '3001.jpg', '4556.jpg', '3023.jpg', '1416.jpg', '7850.jpg', '1871.jpg', '7411.jpg', '1918.jpg', '5889.jpg', '3883.jpg', '7917.jpg', '6226.jpg', '145.jpg', '607.jpg', '6283.jpg', '4264.jpg', '5349.jpg', '5128.jpg', '2511.jpg', '6245.jpg', '3060.jpg', '7691.jpg', '7457.jpg', '6216.jpg', '6017.jpg', '2479.jpg', '3096.jpg', '3446.jpg', '1265.jpg', '742.jpg', '5648.jpg', '2648.jpg', '1403.jpg', '7488.jpg', '655.jpg', '4487.jpg', '4015.jpg', '1002.jpg', '78.jpg', '5581.jpg', '6514.jpg', 

In [113]:
tensor_result=torch.as_tensor(result[0:-1]).flatten()
tensor_result.shape
last=torch.as_tensor(result[-1])
final_tensor=torch.hstack((tensor_result,last))
final=final_tensor.tolist()
from pandas import DataFrame
# your_list = ['item1', 'item2', 'item3',...]
# df = DataFrame (images,final,columns=['id','labels'])
df = DataFrame(final, index =images,columns =['label'])
df.to_csv("sample.csv",index_label='id')
df.head


<bound method NDFrame.head of           label
7084.jpg   3946
7780.jpg   3394
4315.jpg   1844
5162.jpg   2701
5228.jpg   2730
...         ...
4465.jpg    320
6045.jpg    792
4459.jpg   3146
7623.jpg   2790
559.jpg     210

[8000 rows x 1 columns]>

In [114]:
# import the csv file
import pandas as pd
file=pd.read_csv('/content/sample.csv')
# print(file.head)
li=[]
news=[]
# print("The images are",file['id'])
li=file['label']
# print(target_num)
for i in li:
  news.append((target_num[i]))
# print(news)
file['label']=news
file.to_csv('sameercruz.csv')
file.head(5)



,id,label
0,7084.jpg,3334
1,7780.jpg,238
2,4315.jpg,1693
3,5162.jpg,2824
4,5228.jpg,651


In [ ]:
#!kaggle competitions submit -c deep-learning-hw2p2-part-2-classification -f heroine.csv -m "first"